<a href="https://colab.research.google.com/github/pradeepselva204/deep_learning/blob/master/Cuda_programming.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from numba import jit
import math

# This is the function decorator syntax and is equivalent to `hypot = jit(hypot)`.
# The Numba compiler is just a function you can call whenever you want!
@jit
def hypot(x, y):
    # Implementation from https://en.wikipedia.org/wiki/Hypot
    x = abs(x);
    y = abs(y);
    t = min(x, y);
    x = max(x, y);
    t = t / x;
    return x * math.sqrt(1+t*t)

In [2]:
hypot(3.0, 4.0)

5.0

In [3]:
hypot.py_func(3.0, 4.0)

5.0

In [4]:
%timeit hypot.py_func(3.0, 4.0)

The slowest run took 17.01 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 3: 851 ns per loop


In [5]:
%timeit hypot(3.0, 4.0)

The slowest run took 37.41 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 3: 254 ns per loop


In [6]:
%timeit math.hypot(3.0, 4.0)

The slowest run took 171.45 times longer than the fastest. This could mean that an intermediate result is being cached.
10000000 loops, best of 3: 152 ns per loop
